In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world")
world = root_body(mechanism)

RigidBody: "world"

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("joint1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("joint2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("body2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
mechanism

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2

In [6]:
x = MechanismState(Float64, mechanism);

In [7]:
rand!(x)
setdirty!(x)
x.q[joint1]

7-element Array{Float64,1}:
  0.139472 
 -0.0745162
 -0.682726 
  0.713358 
  0.152892 
  0.215082 
  0.434678 

In [8]:
num_positions(x)

8

In [9]:
num_velocities(x)

7

In [10]:
relative_transform(x, joint2.frameAfter, body1.frame)

Transform3D from "after_joint2" to "body1":
rotation: 3.8145928848242336 rad about [0.06807431150940564,-0.8131476913572572,-0.5780629032837891], translation: Vec(0.5123805973122008,0.7780801463660356,0.023584481212894026)


In [11]:
center_of_mass(x)

Point3D in "world": Vec(-0.5214209386457825,-0.7214718582657531,1.6780581093625948)

In [12]:
mass(mechanism)

1.1521318293543166

In [13]:
transform(x, Point3D{Float64}(joint2.frameAfter), root_frame(mechanism))

Point3D in "world": Vec(-0.09043671924011903,0.06186996676874901,1.3197748085186727)

In [14]:
relative_twist(x, body2, body1)

Twist of "after_joint2" w.r.t "after_joint1" in "world":
angular: Vec(-0.03654548765588023,-0.4365883114785414,0.4636554311854492), linear: Vec(0.6048846013026764,-0.006300337918986099,0.04174468265551953)

In [15]:
relative_twist(x, body1, world) + relative_twist(x, body2, body1) + relative_twist(x, world, body2)

Twist of "world" w.r.t "world" in "world":
angular: Vec(0.0,0.0,0.0), linear: Vec(0.0,0.0,0.0)

In [16]:
p = path(mechanism, body2, world)
J = geometric_jacobian(x, p)

GeometricJacobian: body: "world", base: "after_joint2", expressed in "world":
6x7 Array{Float64,2}:
  0.0499024    0.242089   -0.739811   …  -0.0       -0.0       -0.0     
  0.596156     0.611265    0.618746      -0.0       -0.0       -0.0     
 -0.633116     0.753491   -0.26426       -0.0       -0.0       -0.0     
 -0.825963     0.66056    -0.441255       0.242089  -0.739811   0.627752
  0.00860304  -0.209406   -0.0913118      0.611265   0.618746   0.493466
 -0.0570019   -0.0423521   1.02152    …   0.753491  -0.26426   -0.602012

In [17]:
Twist(J, velocity_vector(x, p))

Twist of "world" w.r.t "after_joint2" in "world":
angular: Vec(0.18896752621255805,1.1780122235118737,0.035472503422442916), linear: Vec(-0.16377331031751974,1.509060746300837,-0.04481285267084767)

In [18]:
mass_matrix(x)

7x7 Array{Float64,2}:
  7.90559      -0.140741  -0.5295       …   0.96886      -0.715399 
 -0.140741      6.75333   -2.65077         -0.276356      0.338401 
 -0.5295       -2.65077    1.50945          5.55112e-17  -0.0393211
  1.38778e-17   2.65062   -0.96886         -5.55112e-16   0.102065 
 -2.65062       0.0        0.276356         0.0           0.198972 
  0.96886      -0.276356   5.55112e-17  …   1.15213      -0.126052 
 -0.715399      0.338401  -0.0393211       -0.126052      0.161208 

In [19]:
A = momentum_matrix(x)

MomentumMatrix expressed in "world":
6x7 Array{Float64,2}:
 -2.10454    4.72929   -2.01851    1.80811    0.976588   0.453629   0.370944
 -5.08831   -2.12612    1.04131   -0.920698   1.58906   -0.852005   0.333988
 -3.11928    0.268575   0.283852   0.165983   0.986662  -0.225359   0.342666
 -2.56916   -0.468203   0.439001  -0.278918   0.85236   -0.723253   0.201622
  1.16196   -1.48386    0.421236  -0.704258  -0.712877  -0.568538  -0.1233  
 -0.117188  -2.16359    0.803057  -0.868121   0.304462   0.693597  -0.10021 

In [20]:
Momentum(A, velocity_vector(x))

Momentum expressed in "world":
angular: Vec(2.226770268497883,-4.558179593229836,-1.4337391024538457), linear: Vec(-2.2088810768644542,-1.2271419794727714,-0.3885618854455791)

In [21]:
accel = SpatialAcceleration(joint2.frameAfter, world.frame, joint2.frameAfter, rand(Vec{3, Float64}), rand(Vec{3, Float64}))
transform(x, accel, world.frame)

SpatialAcceleration of "after_joint2" w.r.t "world" in "world":
angular: Vec(0.017472763780686257,-0.8822245037703292,-0.00041093176330946144), linear: Vec(1.1207225615621383,-0.5871971526934621,0.08793776687231661)

In [22]:
bias_acceleration(x, body2)

SpatialAcceleration of "after_joint2" w.r.t "world" in "world":
angular: Vec(-0.561678645807249,0.08891217978207952,0.03944998202058868), linear: Vec(-0.7295186785242682,-0.09114047404199543,0.5871011395268262)

In [23]:
v̇ = [j::Joint => rand(num_velocities(j))::Vector{Float64} for j in joints(mechanism)]
τ = inverse_dynamics(x, v̇)

Dict{RigidBodyDynamics.Joint,Array{Float64,1}} with 2 entries:
  Joint "joint2": Revolut… => [-0.8243700827629163]
  Joint "joint1": Quatern… => [0.7469937573007912,-15.235980580369834,5.5546248…

In [24]:
relative_acceleration(x, body1, body2, v̇)

SpatialAcceleration of "after_joint1" w.r.t "after_joint2" in "world":
angular: Vec(0.606531379800842,0.4469181143392835,-0.6085000906268007), linear: Vec(-0.012863756607477184,0.09887295755434947,-0.6383349105583015)

In [25]:
rand_chain_mechanism(Float64, QuaternionFloating, Revolute{Float64}, Revolute{Float64})

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
      Vertex: body3, Edge: joint3

In [26]:
rand_tree_mechanism(Float64, [QuaternionFloating; [Revolute{Float64} for i = 1 : 10]]...)

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body11, Edge: joint11
  Vertex: body2, Edge: joint2
    Vertex: body3, Edge: joint3
    Vertex: body4, Edge: joint4
      Vertex: body6, Edge: joint6
        Vertex: body8, Edge: joint8
          Vertex: body10, Edge: joint10
        Vertex: body9, Edge: joint9
    Vertex: body5, Edge: joint5
    Vertex: body7, Edge: joint7